In [1]:
#Test random sampling
import random_sampling

print random_sampling.parallel_random_sampling(0.1,["data/part1.tsv","data/part2.tsv"], 1)
print random_sampling.parallel_outbox_sampling((30,30,"*","*"),(60,60,"*","*"),0.1,["data/part1.tsv","data/part2.tsv"], 8)
print random_sampling.parallel_inbox_sampling((30,30,"*","*"),(60,60,"*","*"),0.1,["data/part1.tsv","data/part2.tsv"], 8)


Job done. Completed 2 jobs
[['data/part1.tsv-rand-0.1', 102], ['data/part2.tsv-rand-0.1', 80]]
Job done. Completed 2 jobs
[['data/part1.tsv-outbb-0.1', 40], ['data/part2.tsv-outbb-0.1', 44]]
Job done. Completed 2 jobs
[['data/part1.tsv-inbb-0.1', 10], ['data/part2.tsv-inbb-0.1', 14]]


In [ ]:
# Test uncertainty Sampling
import load_tsv
import numpy
from sklearn import linear_model
import uncertainty_sampling

features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv",[0,1,2],3)
linreg = linear_model.LinearRegression()
linreg.fit(features, truth)
vals = uncertainty_sampling.parallel_reg_uncertainty_sampling(linreg,["data/part1.tsv", "data/part2.tsv"],[0,1,2],3,0.1,1)
print vals

In [ ]:
# Test maximum entropy uncertainty Sampling
import load_tsv
import numpy
from sklearn.svm import SVC
import uncertainty_sampling

features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv",[0,1,2],3)
# Create, fit, predict with the SVM Classifier
svm = SVC(C=1, kernel="linear", probability=True, max_iter=50000)
svm.fit(features, truth)
vals = uncertainty_sampling.parallel_maxent_uncertainty_sampling(svm,["data/part1.tsv", "data/part2.tsv"],[0,1,2],3,0.1,1)
print vals

In [ ]:
#%load parallel_jobs.py
#%load load_tsv.py

In [ ]:
# Test density Sampling
import load_tsv
import numpy
from sklearn.svm import SVC
import density_sampling

features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv",[0,1,2],3)
vals = density_sampling.parallel_density_sampling(["data/part1.tsv", "data/part2.tsv"], "data/densitytest.tsv", [0,1,2], 3, 3, 0.2, 5, 8)    
print vals

In [2]:
# Test query by committee Sampling
import load_tsv
import numpy
from sklearn.svm import SVC
import query_by_committee_sampling

features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv",[0,1,2],3)
# Create, fit, predict with the SVM Classifier
svm = SVC(C=1, kernel="linear", probability=True, max_iter=50000)
svm.fit(features, truth)
#vals = query_by_committee_sampling.query_sampling(([svm], features, truth, 0.1, "data/part1.tsv"))
#features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv", [0,1,2], 3)
vals = query_by_committee_sampling.parallel_query_sampling([svm],["data/part1.tsv", "data/part2.tsv"]  ,[0,1,2],3,0.1,1)
print vals

Job done. Completed 2 jobs
[['data/part1.tsv-query-101', 1017], ['data/part2.tsv-query-98', 983]]


In [ ]:
%matplotlib inline

import matplotlib.pyplot as pyplot
import load_tsv
import numpy
from sklearn.svm import SVC
import uncertainty_sampling

#assumes non-empty array of features
def convert_array(features, truth, idx):
    if idx == -1:
        return numpy.ones(len(features))
    if idx == len(features[0]):
        return truth
    else: #return relevant column
        return features[:,idx]

#if circle_area_feature_idx or color_feature_idx = -1 then choose a default
#if circle_area_feature_idx or color_feature_idx = width of array then choose the truth array
def scatterplot(features, truth, xaxis_feature_idx, yaxis_feature_idx, circle_area_feature_idx, color_feature_idx, axis=None):
    fig, ax = pyplot.subplots()
    ax.grid(True, alpha=0.3)
    pyplot.scatter(convert_array(features, truth, xaxis_feature_idx), 
                       convert_array(features, truth, yaxis_feature_idx),
                       marker='o', 
                       s=convert_array(features, truth, circle_area_feature_idx), 
                       c=convert_array(features, truth, color_feature_idx))
    if axis != None and len(axis) == 4: #in format [xmin,xmax,ymin,ymax]
        pyplot.axis(axis)
    pyplot.gcf().set_size_inches(15, 10)
    pyplot.show()

def boxplot(features, truth, feature_idxs, feature_labels, by_label=None):
    fig, ax = pyplot.subplots()
    data = []
    if by_label == None:
        for idx in feature_idxs:
            data.append(features[:,idx])
        ax.boxplot(data, labels=feature_labels)
    else:
        box_labels = []
        for idx in feature_idxs:
            for j in by_label:
                data.append([features[i,idx] for i in range(len(truth)) if truth[i]==by_label[j]])
                box_labels.append(feature_labels[idx]+"-"+str(by_label[j]))
        ax.boxplot(data, labels=box_labels)
    #pyplot.setp(feature_labels, rotation=45, fontsize=8)
    pyplot.gcf().set_size_inches(15, 10)
    pyplot.show()

def histogram(features, feature_idx, feature_label, bins, axis):
    fig, ax = pyplot.subplots()
    n, bins, patches = pyplot.hist(features[:,feature_idx], bins)
    pyplot.xlabel(feature_label)
    pyplot.gcf().set_size_inches(15, 10)
    pyplot.show()


In [ ]:
import load_tsv
import numpy
from sklearn.svm import SVC
import query_by_committee_sampling
import density_sampling
import random_sampling
import uncertainty_sampling

features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv",[0,1,2],3)
print "Plotting "+"data/part1.tsv"
scatterplot(features, truth, 0,1,2,3, [-100,200,-100,200])

svm = SVC(C=1, kernel="linear", probability=True, max_iter=50000)
svm.fit(features, truth)
print "random"
rand_files = random_sampling.parallel_random_sampling(0.1,["data/part1.tsv","data/part2.tsv"], 1)
print "uncertainty"
unc_files = uncertainty_sampling.parallel_smallmargin_uncertainty_sampling(svm,["data/part1.tsv", "data/part2.tsv"]  ,[0,1,2],3,0.1,1)
print "density"
den_files = density_sampling.parallel_density_sampling(["data/part1.tsv", "data/part2.tsv"], "data/densitytest.tsv", [0,1,2], 3, 3, 0.2, 5, 8)
print "query"
qry_files = query_by_committee_sampling.parallel_query_sampling([svm],["data/part1.tsv", "data/part2.tsv"]  ,[0,1,2],3,0.1,1)

print rand_files, unc_files, den_files, qry_files


if False:
    # Create, fit, predict with the SVM Classifier
    svm = SVC(C=1, kernel="linear", probability=True, max_iter=50000)
    svm.fit(features, truth)
    #features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv", [0,1,2], 3)
    vals = uncertainty_sampling.parallel_smallmargin_uncertainty_sampling(svm,["data/part1.tsv", "data/part2.tsv"]  ,[0,1,2],3,0.1,1)
    for val in vals:
        print "Plotting "+val[0]
        #scatterplot(val[1], truth[val[2]], 0,1,2,3, [-100,200,-100,200])
    
    #features, truth = load_tsv.load_tsv_features_truth("data/part1.tsv", [0,1,2], 3)
    vals = uncertainty_sampling.parallel_maxent_uncertainty_sampling(svm,["data/part1.tsv", "data/part2.tsv"]  ,[0,1,2],3,0.1,1)
    for val in vals:
        print "Plotting "+val[0]
        #scatterplot(val[1], truth[val[2]], 0,1,2,3,[-100,200,-100,200])
        boxplot(val[1],truth[val[2]],[0,1,2],["feature1","feature2","feature3"],[0,1])
        histogram(val[1],0,val[0]+"-feature1",50,None)
